In [1]:
vocab_char_to_num = {
    'A': 0, 'B': 1, 'C': 2, 'D': 3,
    'E': 4, 'F': 5, 'G': 6, 'H': 7,
    'I': 8, 'J': 9, 'K': 10, 'L': 11,
    'M': 12, 'N': 13, 'O': 14, 'P': 15,
    'Q': 16, 'R': 17, 'S': 18, 'T': 19,
    'U': 20, 'V': 21, 'W': 22, 'X': 23,
    'Y': 24, 'Z': 25, ' ': 26,   '.': 27,  
    ',': 28, '?': 29, '!': 30
}

vocab_num_to_char = {}

for key in vocab_char_to_num:
    vocab_num_to_char[vocab_char_to_num[key]] = key

def string_to_numerical(s):
    return [vocab_char_to_num[c] for c in s]

def numerical_to_string(s):
    return [vocab_num_to_char[c] for c in s]


In [2]:
from models import EncoderRNN, DecoderRNN
import torch


# Ceaser - Same Shift Training
encoder_state_dict_c = torch.load('encoder_ceaser_cipher_1024.pth')
decoder_state_dict_c = torch.load('decoder_ceaser_cipher_1024.pth')

encoder_c = EncoderRNN(31, 1024)
decoder_c = DecoderRNN(1024, 31)

encoder_c.load_state_dict(encoder_state_dict_c)
decoder_c.load_state_dict(decoder_state_dict_c)


# Autokey - Same Key Training
encoder_state_dict_a = torch.load('encoder_autokey_cipher_1024.pth')
decoder_state_dict_a = torch.load('decoder_autokey_cipher_1024.pth')

encoder_a = EncoderRNN(31, 1024)
decoder_a = DecoderRNN(1024, 31)

encoder_a.load_state_dict(encoder_state_dict_a)
decoder_a.load_state_dict(decoder_state_dict_a)


# Vigenere - Same Key training
encoder_state_dict_v = torch.load('encoder_vigenere_hard_cipher_1024.pth')
decoder_state_dict_v = torch.load('decoder_vigenere_hard_cipher_1024.pth')

encoder_v = EncoderRNN(31, 1024)
decoder_v = DecoderRNN(1024, 31)

encoder_v.load_state_dict(encoder_state_dict_v)
decoder_v.load_state_dict(decoder_state_dict_v)

device = 'cuda:0'

models = [encoder_v, encoder_c, encoder_a, decoder_v, decoder_c, decoder_a]

for m in models:
    m.to(device)
    m.eval()

In [3]:
from ciphers import VigenereCipher 
from ciphers import CaesarCipher
from ciphers import AutokeyCipher


v = VigenereCipher('RWPFUQBZIRASDLJFUTBSMTRYSFYRMWPCVJDHGFJSDMFJNEDFGV')
a = AutokeyCipher('FOOBAR')
c = CaesarCipher(4)


clear_text = "I LOVE IT SO MUCH THIS IS THE BEST TOY EVER GREAT"

cipher_a = a.cipher_text(clear_text)
clear_a = a.original_text(cipher_a)

cipher_v = v.cipher_text(clear_text)
clear_v = v.original_text(cipher_v)

cipher_c = c.cipher_text(clear_text)
clear_c = c.original_text(cipher_c)


print(cipher_a, clear_a)
print(cipher_v, clear_v)
print(cipher_c, clear_c)

NJZPVVDD!JISVUI,ZJOT,UCDGC.Z!DT!G !OP,NXQXATBVZEF I LOVE IT SO MUCH THIS IS THE BEST TOY EVER GREAT
ZR TKU.C.MSB?X?H.OUZUGMBFAMYQRQGI,? U?EWYQWETVHFZ I LOVE IT SO MUCH THIS IS THE BEST TOY EVER GREAT
M!PSZI!MX!WS!QYGL!XLMW!MW!XLI!FIWX!XS,!IZIV!KVIEX I LOVE IT SO MUCH THIS IS THE BEST TOY EVER GREAT


In [4]:

# It was trained at a shift of 4
cipher = c.cipher_text(clear_text)
print(clear_text)
print(cipher)
cipher = torch.tensor(string_to_numerical(cipher)).view(1, -1).to(device)


encoder_outputs, encoder_hidden = encoder_c(cipher)
decoder_outputs, decoder_hidden, decoder_attn = decoder_c(encoder_outputs, encoder_hidden)

_, topi = decoder_outputs.topk(1)
decoded_ids = topi.squeeze()

print("".join(numerical_to_string(decoded_ids.tolist())))


I LOVE IT SO MUCH THIS IS THE BEST TOY EVER GREAT
M!PSZI!MX!WS!QYGL!XLMW!MW!XLI!FIWX!XS,!IZIV!KVIEX
I LOVE IT SO MUCH THIS IS THE BEST TO EVERY GET TR


In [5]:
cipher = a.cipher_text(clear_text)
print(clear_text)
print(cipher)
cipher = torch.tensor(string_to_numerical(cipher)).view(1, -1).to(device)


encoder_outputs, encoder_hidden = encoder_a(cipher)
decoder_outputs, decoder_hidden, decoder_attn = decoder_a(encoder_outputs, encoder_hidden)

_, topi = decoder_outputs.topk(1)
decoded_ids = topi.squeeze()

print("".join(numerical_to_string(decoded_ids.tolist())))

I LOVE IT SO MUCH THIS IS THE BEST TOY EVER GREAT
NJZPVVDD!JISVUI,ZJOT,UCDGC.Z!DT!G !OP,NXQXATBVZEF
 THE APP IS A GREAT AND IT WAS A GREAT AND IT WAS 


In [6]:
cipher = v.cipher_text(clear_text)
print(clear_text)
print(cipher)
cipher = torch.tensor(string_to_numerical(cipher)).view(1, -1).to(device)


encoder_outputs, encoder_hidden = encoder_v(cipher)
decoder_outputs, decoder_hidden, decoder_attn = decoder_v(encoder_outputs, encoder_hidden)

_, topi = decoder_outputs.topk(1)
decoded_ids = topi.squeeze()

print("".join(numerical_to_string(decoded_ids.tolist())))

I LOVE IT SO MUCH THIS IS THE BEST TOY EVER GREAT
ZR TKU.C.MSB?X?H.OUZUGMBFAMYQRQGI,? U?EWYQWETVHFZ
I LOVE IT SO EASY TO USE THAT I ENDED UP UP SO MUC
